# 6장. 에이전트와 도구

In [ ]:
!pip install langchain langchain-community langchain-openai langgraph

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

---
## 코드 6-1~6-2 기본 에이전트: 도구를 사용하는 AI

# 기본 에이전트: 도구를 사용하는 AI

이 노트북에서는 **LangGraph**를 사용하여 **도구(Tool)를 사용하는 에이전트**를 만듭니다.

## 에이전트(Agent)란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    에이전트의 개념                                  │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 챗봇:                                                       │
│   사용자 질문 → LLM → 답변                                        │
│   (LLM이 알고 있는 정보로만 답변)                                  │
│                                                                    │
│   에이전트:                                                        │
│   사용자 질문 → LLM → 도구 호출 → 결과 확인 → 답변                │
│   (외부 도구를 사용해서 실시간 정보 획득 가능!)                    │
│                                                                    │
│   에이전트 = LLM + 도구 사용 능력                                  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 이번에 만들 시스템 (ReAct 패턴)

```
┌────────────────────────────────────────────────────────────────────┐
│                    ReAct 에이전트 아키텍처                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│                    ┌─────────┐                                     │
│                    │  START  │                                     │
│                    └────┬────┘                                     │
│                         │                                          │
│                         ▼                                          │
│                ┌────────────────┐                                  │
│                │     model      │ ◄─────────────┐                  │
│                │  (LLM + Tools) │               │                  │
│                └────────┬───────┘               │                  │
│                         │                       │                  │
│              tools_condition?                   │                  │
│                    ╱    ╲                       │                  │
│            도구 필요   도구 불필요              │                  │
│                 ╱          ╲                    │                  │
│                ▼            ▼                   │                  │
│         ┌──────────┐   ┌─────────┐             │                  │
│         │  tools   │   │   END   │             │                  │
│         │ (도구실행)│   └─────────┘             │                  │
│         └─────┬────┘                           │                  │
│               │                                │                  │
│               └────────────────────────────────┘                  │
│                      (결과를 다시 model로)                         │
│                                                                    │
│   ReAct = Reason + Act (추론하고 행동하기)                         │
│   LLM이 스스로 도구가 필요한지 판단하고, 필요하면 사용함           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 도구(Tool) 정의

에이전트가 사용할 도구들을 정의합니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    도구(Tool)의 구성요소                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   @tool 데코레이터로 함수를 도구로 변환                            │
│                                                                    │
│   필수 요소:                                                       │
│   • 함수 이름: LLM이 도구를 식별하는 데 사용                       │
│   • docstring: LLM이 도구의 용도를 이해하는 데 사용 (매우 중요!)   │
│   • 파라미터: 도구에 전달할 입력값                                 │
│   • 반환값: 도구 실행 결과                                         │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
import ast
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

# 도구 1: 계산기
@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)

# 도구 2: 웹 검색 (DuckDuckGo)
search = DuckDuckGoSearchRun()

# 도구 리스트
tools = [search, calculator]

print("✅ 도구 정의 완료")
print(f"   - search: 웹에서 정보 검색")
print(f"   - calculator: 수학 계산")

In [ ]:
# 도구 테스트
print("=== 도구 테스트 ===")
print(f"계산기: 2 + 3 * 4 = {calculator.invoke('2 + 3 * 4')}")
print(f"\n검색: {search.invoke('Python programming')[:200]}...")

# 3. LLM에 도구 연결 (bind_tools)

LLM이 도구를 사용할 수 있도록 연결합니다.

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델에 도구 연결
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)

print("✅ LLM에 도구 연결 완료")
print("   bind_tools()로 LLM이 도구를 호출할 수 있게 됨")

# 4. State 정의

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    에이전트의 상태
    
    messages: 대화 기록 (사용자 질문, LLM 응답, 도구 결과 등)
    """
    messages: Annotated[list, add_messages]

print("✅ State 정의 완료")

# 5. 노드 정의

```
┌────────────────────────────────────────────────────────────────────┐
│                    두 가지 핵심 노드                               │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ model_node (직접 구현)                                       │
│      • LLM을 호출하여 응답 생성                                    │
│      • 도구가 필요하면 tool_calls 포함된 응답 반환                 │
│                                                                    │
│   2️⃣ ToolNode (LangGraph 제공)                                    │
│      • tool_calls를 받아서 실제로 도구 실행                        │
│      • 결과를 ToolMessage로 반환                                   │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langgraph.prebuilt import ToolNode

def model_node(state: State) -> State:
    """
    LLM 호출 노드
    
    1. 현재 대화 기록을 LLM에 전달
    2. LLM이 응답 생성 (도구 호출 포함 가능)
    3. 응답을 State에 추가
    """
    res = model.invoke(state['messages'])
    return {'messages': res}

# LangGraph가 제공하는 ToolNode 사용
tool_node = ToolNode(tools)

print("✅ 노드 정의 완료")
print("   - model_node: LLM 호출")
print("   - tool_node: 도구 실행 (ToolNode 사용)")

# 6. 그래프 구성

```
┌────────────────────────────────────────────────────────────────────┐
│                    tools_condition 이해하기                        │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   tools_condition은 LangGraph가 제공하는 조건 함수                 │
│                                                                    │
│   LLM 응답에 tool_calls가 있으면:                                  │
│   → "tools" 노드로 이동 (도구 실행)                                │
│                                                                    │
│   tool_calls가 없으면:                                             │
│   → "__end__" 로 이동 (종료)                                       │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition

# 그래프 빌더 생성
builder = StateGraph(State)

# 노드 추가
builder.add_node('model', model_node)
builder.add_node('tools', tool_node)

# 엣지 추가
builder.add_edge(START, 'model')           # 시작 → 모델
builder.add_conditional_edges('model', tools_condition)  # 조건부 분기
builder.add_edge('tools', 'model')         # 도구 → 모델 (루프)

# 그래프 컴파일
graph = builder.compile()

print("✅ 에이전트 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 에이전트 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 7. 에이전트 실행

In [ ]:
from langchain_core.messages import HumanMessage

# 테스트: 검색이 필요한 질문
input_data = {
    'messages': [
        HumanMessage(
            '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'
        )
    ]
}

print("=== 에이전트 실행 (스트림) ===")
print(f"질문: {input_data['messages'][0].content}\n")

for chunk in graph.stream(input_data):
    for node_name, node_output in chunk.items():
        print(f"--- {node_name} ---")
        if 'messages' in node_output:
            for msg in node_output['messages'] if isinstance(node_output['messages'], list) else [node_output['messages']]:
                print(f"  유형: {type(msg).__name__}")
                if hasattr(msg, 'content') and msg.content:
                    print(f"  내용: {str(msg.content)[:200]}..." if len(str(msg.content)) > 200 else f"  내용: {msg.content}")
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    print(f"  도구 호출: {msg.tool_calls}")
        print()

In [ ]:
# 최종 결과만 확인
result = graph.invoke(input_data)

print("=== 최종 응답 ===")
print(result['messages'][-1].content)

# 8. 다양한 질문으로 테스트

In [ ]:
test_questions = [
    "123 * 456 + 789는 얼마인가요?",        # 계산기 필요
    "오늘 서울 날씨는 어떤가요?",           # 검색 필요
    "안녕하세요, 반갑습니다!",              # 도구 불필요
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"질문: {question}")
    print("="*60)
    
    result = graph.invoke({'messages': [HumanMessage(question)]})
    
    # 어떤 도구가 사용되었는지 확인
    tools_used = []
    for msg in result['messages']:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            tools_used.extend([tc['name'] for tc in msg.tool_calls])
    
    print(f"\n사용된 도구: {tools_used if tools_used else '없음'}")
    print(f"\n응답: {result['messages'][-1].content[:300]}..." if len(result['messages'][-1].content) > 300 else f"\n응답: {result['messages'][-1].content}")

---

## 정리: 기본 에이전트 (ReAct 패턴)

### 아키텍처

```
┌─────────────────────────────────────────────────────────────────────┐
│                    ReAct 에이전트 동작 흐름                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. 사용자 질문 입력                                               │
│   2. LLM이 질문 분석                                                │
│   3. 도구가 필요하면 tool_calls 생성                                │
│   4. ToolNode가 도구 실행                                           │
│   5. 결과를 다시 LLM에 전달                                         │
│   6. LLM이 최종 답변 생성 (또는 추가 도구 호출)                     │
│   7. 도구 호출이 없으면 종료                                        │
│                                                                     │
│   핵심: LLM이 스스로 도구 사용 여부를 결정!                         │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 도구 정의
@tool
def my_tool(query: str) -> str:
    '''도구 설명 (LLM이 이해할 수 있게)'''
    return result

# 2. LLM에 도구 연결
model = ChatOllama(model='llama3.2').bind_tools(tools)

# 3. 조건부 엣지로 루프 구성
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')  # 결과를 다시 모델로
```

### 핵심 컴포넌트

| 컴포넌트 | 역할 | 제공 |
|----------|------|------|
| **@tool** | 함수를 도구로 변환 | LangChain |
| **bind_tools()** | LLM에 도구 연결 | LangChain |
| **ToolNode** | 도구 실행 노드 | LangGraph |
| **tools_condition** | 도구 호출 여부 판단 | LangGraph |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1).bind_tools(tools)

# 변경
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)
```

## 다음 단계

**첫 번째 도구를 강제로 호출**하는 방법을 배웁니다. (03-04번 노트북)

---
## 코드 6-3~6-4 첫 번째 도구 강제 호출

# 첫 번째 도구 강제 호출

이 노트북에서는 **첫 번째 도구를 강제로 호출**하는 에이전트를 만듭니다.

## 왜 첫 번째 도구를 강제할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    도구 강제 호출의 필요성                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 에이전트:                                                   │
│   "한국의 수도는 어디인가요?" → LLM이 바로 답변 (도구 안 씀)       │
│   (LLM이 이미 알고 있으면 도구를 사용하지 않음)                    │
│                                                                    │
│   문제:                                                            │
│   • 최신 정보가 필요한 질문인데 LLM의 오래된 지식으로 답변         │
│   • 항상 검색을 해야 하는 시스템인데 LLM이 검색 안 함              │
│                                                                    │
│   해결:                                                            │
│   첫 번째 노드에서 무조건 검색 도구를 호출하도록 강제!             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처 비교

```
┌────────────────────────────────────────────────────────────────────┐
│                    일반 vs 강제 호출 아키텍처                       │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   [일반 에이전트]              [강제 호출 에이전트]                │
│                                                                    │
│   START                        START                               │
│     │                            │                                 │
│     ▼                            ▼                                 │
│   model ─────┐               first_model ← 강제로 검색 호출        │
│     │        │                   │                                 │
│   (조건)    END                  ▼                                 │
│     │                         tools                                │
│     ▼                            │                                 │
│   tools                          ▼                                 │
│     │                         model ─────┐                         │
│     └────────┘                   │       │                         │
│                                (조건)   END                        │
│                                  │                                 │
│                                  ▼                                 │
│                               tools                                │
│                                  │                                 │
│                                  └───────┘                         │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 도구 정의

In [ ]:
import ast
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]

print("✅ 도구 정의 완료")

# 3. LLM 설정

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)

print("✅ LLM 설정 완료")

# 4. State 정의

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

print("✅ State 정의 완료")

# 5. 노드 정의 (핵심: first_model)

```
┌────────────────────────────────────────────────────────────────────┐
│                    ToolCall 직접 생성하기                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반적으로 LLM이 tool_calls를 생성하지만,                        │
│   우리가 직접 ToolCall을 만들어서 강제로 도구 호출 가능!           │
│                                                                    │
│   ToolCall 구성요소:                                               │
│   • name: 호출할 도구 이름                                         │
│   • args: 도구에 전달할 인자 (딕셔너리)                            │
│   • id: 고유 식별자 (uuid 사용)                                    │
│                                                                    │
│   예시:                                                            │
│   ToolCall(                                                        │
│       name='duckduckgo_search',                                    │
│       args={'query': '사용자 질문'},                               │
│       id='abc123'                                                  │
│   )                                                                │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from uuid import uuid4
from langchain_core.messages import AIMessage, HumanMessage, ToolCall

def model_node(state: State) -> State:
    """
    일반 LLM 호출 노드 (두 번째 이후 사용)
    """
    res = model.invoke(state['messages'])
    return {'messages': res}


def first_model(state: State) -> State:
    """
    첫 번째 노드: 검색 도구 강제 호출
    
    1. 사용자 질문을 가져옴
    2. ToolCall을 직접 생성 (검색 도구)
    3. AIMessage에 tool_calls 포함하여 반환
    """
    # 사용자 질문 가져오기
    query = state['messages'][-1].content
    
    # 검색 도구 호출을 직접 생성
    search_tool_call = ToolCall(
        name='duckduckgo_search',  # 도구 이름
        args={'query': query},      # 도구 인자
        id=uuid4().hex              # 고유 ID
    )
    
    # tool_calls가 포함된 AIMessage 반환
    return {
        'messages': AIMessage(
            content='',  # 내용은 비워둠
            tool_calls=[search_tool_call]  # 도구 호출 정보
        )
    }

print("✅ 노드 정의 완료")
print("   - first_model: 첫 번째로 검색 강제 호출")
print("   - model_node: 이후 LLM 판단에 따라 도구 사용")

# 6. 그래프 구성

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)

# 노드 추가 (first_model 추가됨)
builder.add_node('first_model', first_model)  # 첫 번째 노드
builder.add_node('model', model_node)         # 일반 모델 노드
builder.add_node('tools', ToolNode(tools))

# 엣지 추가
builder.add_edge(START, 'first_model')         # 시작 → first_model
builder.add_edge('first_model', 'tools')       # first_model → tools (항상)
builder.add_conditional_edges('model', tools_condition)  # model은 조건부
builder.add_edge('tools', 'model')             # tools → model

graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 7. 에이전트 실행

In [ ]:
# 테스트 실행
input_data = {
    'messages': [
        HumanMessage(
            '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'
        )
    ]
}

print("=== 강제 검색 에이전트 실행 ===")
print(f"질문: {input_data['messages'][0].content}\n")

for chunk in graph.stream(input_data):
    for node_name, node_output in chunk.items():
        print(f"--- {node_name} ---")
        if node_name == 'first_model':
            print("  → 검색 도구 강제 호출!")
        print()

In [ ]:
# 최종 결과
result = graph.invoke(input_data)

print("=== 최종 응답 ===")
print(result['messages'][-1].content)

# 8. 일반 에이전트와 비교

LLM이 이미 알 법한 질문으로 테스트해봅니다.

In [ ]:
# 일반 에이전트 만들기 (비교용)
normal_builder = StateGraph(State)
normal_builder.add_node('model', model_node)
normal_builder.add_node('tools', ToolNode(tools))
normal_builder.add_edge(START, 'model')
normal_builder.add_conditional_edges('model', tools_condition)
normal_builder.add_edge('tools', 'model')
normal_graph = normal_builder.compile()

print("✅ 일반 에이전트 (비교용) 생성 완료")

In [ ]:
# 비교 테스트
test_question = "한국의 수도는 어디인가요?"

print("=" * 60)
print(f"질문: {test_question}")
print("=" * 60)

# 일반 에이전트
print("\n[일반 에이전트]")
normal_result = normal_graph.invoke({'messages': [HumanMessage(test_question)]})
normal_tools_used = [tc['name'] for msg in normal_result['messages'] 
                     if hasattr(msg, 'tool_calls') and msg.tool_calls 
                     for tc in msg.tool_calls]
print(f"  사용된 도구: {normal_tools_used if normal_tools_used else '없음'}")

# 강제 검색 에이전트
print("\n[강제 검색 에이전트]")
forced_result = graph.invoke({'messages': [HumanMessage(test_question)]})
forced_tools_used = [tc['name'] for msg in forced_result['messages'] 
                     if hasattr(msg, 'tool_calls') and msg.tool_calls 
                     for tc in msg.tool_calls]
print(f"  사용된 도구: {forced_tools_used if forced_tools_used else '없음'}")

print("\n→ 강제 검색 에이전트는 항상 검색부터 시작!")

---

## 정리: 첫 번째 도구 강제 호출

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    ToolCall 직접 생성                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   from langchain_core.messages import ToolCall                      │
│   from uuid import uuid4                                            │
│                                                                     │
│   tool_call = ToolCall(                                             │
│       name='도구_이름',          # 호출할 도구                      │
│       args={'key': 'value'},     # 전달할 인자                      │
│       id=uuid4().hex             # 고유 ID                          │
│   )                                                                 │
│                                                                     │
│   # AIMessage에 포함하여 반환                                       │
│   AIMessage(content='', tool_calls=[tool_call])                     │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
def first_model(state: State) -> State:
    query = state['messages'][-1].content
    
    # 도구 호출 직접 생성
    search_tool_call = ToolCall(
        name='duckduckgo_search',
        args={'query': query},
        id=uuid4().hex
    )
    
    return {
        'messages': AIMessage(content='', tool_calls=[search_tool_call])
    }

# 그래프에서 first_model → tools 직접 연결 (조건 없이)
builder.add_edge('first_model', 'tools')
```

### 사용 시나리오

| 시나리오 | 설명 |
|----------|------|
| 뉴스 검색 봇 | 항상 최신 뉴스 검색 후 답변 |
| 실시간 정보 | 날씨, 주가 등 항상 검색 필요 |
| RAG 시스템 | 항상 문서 검색 후 답변 |
| 팩트체크 | 답변 전 사실 확인 필수 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(temperature=0.1).bind_tools(tools)

# 변경
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)
```

## 다음 단계

**많은 도구 중에서 관련 도구만 선택**하는 방법을 배웁니다. (05-06번 노트북)

---
## 코드 6-5~6-6 많은 도구 관리: 동적 도구 선택

# 많은 도구 관리: 동적 도구 선택

이 노트북에서는 **많은 도구 중에서 관련 도구만 선택**하는 에이전트를 만듭니다.

## 왜 동적 도구 선택이 필요할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    많은 도구의 문제점                               │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   도구가 많아지면:                                                 │
│                                                                    │
│   1️⃣ 컨텍스트 윈도우 낭비                                         │
│      • 모든 도구 설명을 LLM에 전달해야 함                          │
│      • 토큰 비용 증가, 응답 속도 저하                              │
│                                                                    │
│   2️⃣ LLM 혼란                                                     │
│      • 비슷한 도구들 중 선택 어려움                                │
│      • 잘못된 도구 선택 가능성 증가                                │
│                                                                    │
│   해결책:                                                          │
│   질문과 관련된 도구만 선택 → 선택된 도구만 LLM에 전달             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    동적 도구 선택 아키텍처                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   START                                                            │
│     │                                                              │
│     ▼                                                              │
│   ┌──────────────┐                                                 │
│   │ select_tools │  ← 도구 설명을 벡터 검색하여 관련 도구 선택     │
│   └──────┬───────┘                                                 │
│          │                                                         │
│          ▼                                                         │
│   ┌──────────────┐                                                 │
│   │    model     │  ← 선택된 도구만 bind_tools                     │
│   └──────┬───────┘                                                 │
│          │                                                         │
│      (조건부)─────────────┐                                        │
│          │                │                                        │
│          ▼                ▼                                        │
│   ┌──────────────┐   ┌─────────┐                                   │
│   │    tools     │   │   END   │                                   │
│   └──────┬───────┘   └─────────┘                                   │
│          │                                                         │
│          └───────────────────────────────────────┐                 │
│                                                  │                 │
│                                              model                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 도구 정의

In [ ]:
import ast
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]

print("✅ 도구 정의 완료")
for t in tools:
    print(f"   - {t.name}: {t.description[:50]}...")

# 3. 도구 Retriever 생성

도구 설명을 벡터화하여 검색할 수 있게 합니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    도구 Retriever 동작                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1. 각 도구의 설명(description)을 Document로 변환                 │
│   2. Document를 벡터로 임베딩하여 저장                             │
│   3. 사용자 질문으로 유사한 도구 설명 검색                         │
│   4. 검색된 도구만 LLM에 전달                                      │
│                                                                    │
│   예시:                                                            │
│   질문: "123 + 456은?"                                            │
│   → "계산기" 설명과 유사도 높음                                    │
│   → calculator 도구 선택                                           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain_core.documents import Document
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings

# 임베딩 모델
embeddings = OllamaEmbeddings(model='nomic-embed-text')

# 도구 설명을 Document로 변환
tool_documents = [
    Document(
        page_content=tool.description,  # 도구 설명
        metadata={'name': tool.name}     # 도구 이름
    )
    for tool in tools
]

print("=== 도구 Document ===")
for doc in tool_documents:
    print(f"  {doc.metadata['name']}: {doc.page_content}")

In [ ]:
# 벡터 스토어 생성 및 Retriever 변환
tools_retriever = InMemoryVectorStore.from_documents(
    tool_documents,
    embeddings
).as_retriever()

print("✅ 도구 Retriever 생성 완료")

In [ ]:
# Retriever 테스트
test_queries = [
    "100 + 200을 계산해줘",
    "오늘 뉴스 검색해줘",
]

print("=== Retriever 테스트 ===")
for query in test_queries:
    results = tools_retriever.invoke(query)
    print(f"\n질문: {query}")
    print(f"선택된 도구: {[doc.metadata['name'] for doc in results]}")

# 4. LLM 설정

In [ ]:
from langchain_ollama import ChatOllama

# 도구 없이 기본 모델 생성 (나중에 동적으로 bind_tools)
model = ChatOllama(model='llama3.2', temperature=0.1)

print("✅ LLM 설정 완료")
print("   도구는 select_tools 노드 후에 동적으로 연결됨")

# 5. State 정의 (확장)

선택된 도구 목록을 저장하기 위해 State를 확장합니다.

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    확장된 State
    
    messages: 대화 기록
    selected_tools: 선택된 도구 이름 목록 (새로 추가!)
    """
    messages: Annotated[list, add_messages]
    selected_tools: list[str]  # 선택된 도구 이름

print("✅ State 정의 완료")
print("   - messages: 대화 기록")
print("   - selected_tools: 선택된 도구 목록")

# 6. 노드 정의

In [ ]:
def select_tools(state: State) -> State:
    """
    도구 선택 노드
    
    1. 사용자 질문 가져오기
    2. Retriever로 관련 도구 검색
    3. 선택된 도구 이름 반환
    """
    query = state['messages'][-1].content
    tool_docs = tools_retriever.invoke(query)
    
    return {
        'selected_tools': [doc.metadata['name'] for doc in tool_docs]
    }


def model_node(state: State) -> State:
    """
    LLM 호출 노드 (동적 도구 연결)
    
    1. selected_tools에 있는 도구만 필터링
    2. 필터링된 도구로 bind_tools
    3. LLM 호출
    """
    # 선택된 도구만 필터링
    selected_tools = [
        tool for tool in tools 
        if tool.name in state['selected_tools']
    ]
    
    # 동적으로 도구 연결 후 호출
    res = model.bind_tools(selected_tools).invoke(state['messages'])
    
    return {'messages': res}


print("✅ 노드 정의 완료")
print("   - select_tools: 관련 도구 선택")
print("   - model_node: 선택된 도구만 사용하여 LLM 호출")

# 7. 그래프 구성

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)

# 노드 추가
builder.add_node('select_tools', select_tools)  # 도구 선택 노드
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))

# 엣지 추가
builder.add_edge(START, 'select_tools')         # 시작 → 도구 선택
builder.add_edge('select_tools', 'model')       # 도구 선택 → 모델
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 8. 에이전트 실행

In [ ]:
from langchain_core.messages import HumanMessage

# 테스트
input_data = {
    'messages': [
        HumanMessage(
            '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'
        )
    ]
}

print("=== 동적 도구 선택 에이전트 실행 ===")
print(f"질문: {input_data['messages'][0].content}\n")

for chunk in graph.stream(input_data):
    for node_name, node_output in chunk.items():
        print(f"--- {node_name} ---")
        if 'selected_tools' in node_output:
            print(f"  선택된 도구: {node_output['selected_tools']}")
        print()

In [ ]:
# 최종 결과
result = graph.invoke(input_data)

print("=== 최종 결과 ===")
print(f"선택된 도구: {result.get('selected_tools', [])}")
print(f"\n응답: {result['messages'][-1].content}")

# 9. 다양한 질문으로 테스트

In [ ]:
test_questions = [
    "123 * 456은 얼마인가요?",              # 계산기
    "오늘 한국의 날씨는 어떤가요?",         # 검색
    "파이썬에서 리스트를 정렬하는 방법",    # 검색
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"질문: {question}")
    print("="*60)
    
    result = graph.invoke({'messages': [HumanMessage(question)]})
    
    print(f"\n선택된 도구: {result.get('selected_tools', [])}")
    print(f"\n응답: {result['messages'][-1].content[:200]}..." if len(result['messages'][-1].content) > 200 else f"\n응답: {result['messages'][-1].content}")

---

## 정리: 동적 도구 선택

### 아키텍처

```
┌─────────────────────────────────────────────────────────────────────┐
│                    동적 도구 선택 흐름                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   도구 설명 → 임베딩 → 벡터 DB 저장                                │
│                                                                     │
│   사용자 질문 → 벡터 검색 → 관련 도구 선택                         │
│                                                                     │
│   선택된 도구만 LLM에 전달 → 효율적인 처리                         │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 도구 Retriever 생성
tool_documents = [
    Document(tool.description, metadata={'name': tool.name})
    for tool in tools
]
tools_retriever = InMemoryVectorStore.from_documents(
    tool_documents, embeddings
).as_retriever()

# 2. 도구 선택 노드
def select_tools(state):
    query = state['messages'][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {'selected_tools': [doc.metadata['name'] for doc in tool_docs]}

# 3. 동적 bind_tools
def model_node(state):
    selected = [t for t in tools if t.name in state['selected_tools']]
    res = model.bind_tools(selected).invoke(state['messages'])
    return {'messages': res}
```

### 장점

| 장점 | 설명 |
|------|------|
| **토큰 절약** | 필요한 도구 설명만 전달 |
| **응답 속도** | 컨텍스트 줄어서 빨라짐 |
| **정확도** | LLM이 선택할 도구가 적어져 혼란 감소 |
| **확장성** | 도구가 수십 개여도 효율적 처리 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
embeddings = OpenAIEmbeddings()
model = ChatOpenAI(temperature=0.1)

# 변경
embeddings = OllamaEmbeddings(model='nomic-embed-text')
model = ChatOllama(model='llama3.2', temperature=0.1)
```

## ch06 요약: 에이전트 패턴

| 패턴 | 용도 | 핵심 |
|------|------|------|
| **기본 ReAct** | LLM이 도구 사용 판단 | tools_condition |
| **강제 호출** | 항상 특정 도구 먼저 | ToolCall 직접 생성 |
| **동적 선택** | 많은 도구 효율화 | 도구 Retriever |

## 다음 단계

**ch07에서는** Reflection, Subgraph, Supervisor 등 **고급 에이전트 패턴**을 배웁니다.